# Установим psycopg2, если это необходимо (или другие библиотеки)

In [ ]:
pip install psycopg2

# Подключим все необходимые библиотеки для работы

In [ ]:
# Библиотека ConfigParser в ядре Python включает в себя модуль, под названием configparser, 
# который вы можете использовать для создания и работы с файлами конфигурации. 
from configparser import ConfigParser

# Библиотека Psycopg2 для подключения к PostgreSQL, выполнения SQL-запросов и других операций с базой данных
import psycopg2
import psycopg2.extras as psql_extras

# Модуль typing в Python для аннотации типов.
from typing import Dict, List

# Библиотека на языке Python для обработки и анализа данных.
import pandas as pd

print('Импорт библиотек завершен')

# Создадим функции для дальнейшей работы с базой данных

### Функция load_connection_info принимает на вход имя файла .ini, содержащего информацию о подключении к базе данных PostgreSQL, и возвращает словарь с этой информацией.

In [ ]:
def load_connection_info(ini_filename: str) -> Dict[str, str]:
    #Внутри функции используется стандартный модуль ConfigParser для чтения файла .ini.
    parser = ConfigParser()
    parser.read(ini_filename)
    # С помощью метода items() объекта парсера мы получаем список кортежей, 
    # содержащих пары "ключ-значение" из раздела "postgresql" файла .ini.
    conn_info = {param[0]: param[1] for param in parser.items("postgresql")}
    return conn_info

### Функция create_db принимает на вход словарь conn_info с информацией о подключении к базе данных PostgreSQL и создает новую базу данных

In [ ]:
# Функция create_db принимает на вход словарь conn_info с информацией 
# о подключении к базе данных PostgreSQL и создает новую базу данных

def create_db(conn_info: Dict[str, str],) -> None:
    # Cоздаем строку подключения к базе данных с помощью значений из словаря conn_info
    psql_connection_string = f"host={conn_info['host']} port={conn_info['port']} user={conn_info['user']} password={conn_info['password']}"
    # Устанавливаем соединение с базой данных PostgreSQL
    conn = psycopg2.connect(psql_connection_string)
    # Создаем курсор для выполнения SQL-запросов
    cur = conn.cursor()
    # операция "CREATE DATABASE" требует автоматического завершения транзакции (commits)
    conn.autocommit = True
    # Формируем SQL-запрос на создание базы данных
    sql_query = f"CREATE DATABASE {conn_info['database']}"

    try:
        # Выполняем SQL-запрос на создание базы данных
        cur.execute(sql_query)
    except Exception as e:
        # Если произошла ошибка, выводим ее сообщение и SQL-запрос, который вызвал ошибку
        print(f"{type(e).__name__}: {e}")
        print(f"Query: {cur.query}")
        cur.close()
    else:
        # Возвращаем отмену автозавершения транзакций
        conn.autocommit = False

### Функция create_table принимает на вход SQL-запрос sql_query,соединение с базой данных conn и курсор для выполнения SQL-запросов cur

In [ ]:
def create_table(sql_query: str,conn: psycopg2.extensions.connection, cur:psycopg2.extensions.cursor) -> None:
    try:
        # В блоке try-except мы выполняем SQL-запрос на создание таблицы и обрабатываем возможные ошибки. 
        # Если произошла ошибка, мы выводим сообщение об ошибке и SQL-запрос, 
        # который вызвал ошибку, а затем откатываем транзакцию и закрываем курсор
        cur.execute(sql_query)
    except Exception as e:
        print(f"{type(e).__name__}: {e}")
        print(f"Query: {cur.query}")
        conn.rollback()
        cur.close()
    else:
        # Чтобы изменения вступили в силу, их необходимо зафиксировать в базе данных
        conn.commit()

### Функция load_connection_info принимает на вход имя файла ini_filename, содержащего информацию о подключении к базе данных PostgreSQL.

In [ ]:
def load_connection_info(ini_filename: str) -> Dict[str, str]:
    # Мы создаем объект парсера ConfigParser и
    # считываем информацию из файла ini_filename
    parser = ConfigParser()
    parser.read(ini_filename)
    # Создадим словарь переменных, хранящихся в разделе "postgresql" файла .ini
    conn_info = {param[0]: param[1] for param in parser.items("postgresql")}
    # Функция возвращает словарь conn_info с информацией 
    # о подключении к базе данных PostgreSQL
    return conn_info

### Функция insert_data принимает на вход следующие параметры:
### query - SQL-запрос для вставки данных в таблицу
### conn - соединение с базой данных PostgreSQL
### cur - курсор для выполнения SQL-запросов
### df - DataFrame, содержащий данные для вставки в таблицу
### page_size - размер страницы при вставке данных

In [ ]:
def insert_data(
    query: str,
    conn: psycopg2.extensions.connection,
    cur: psycopg2.extensions.cursor,
    df: pd.DataFrame,
    page_size: int
) -> None:
    
    # Создаем список кортежей data_tuples из строк DataFrame df, 
    # преобразованных в numpy-массивы и затем в кортежи
    data_tuples = [tuple(row.to_numpy()) for index, row in df.iterrows()]

    try:
        # Выполняем массовую вставку данных с помощью метода execute_values из модуля psycopg2.extras
        #В качестве аргументов передаем курсор, SQL-запрос, список кортежей data_tuples и размер страницы page_size
        psql_extras.execute_values(
            cur, query, data_tuples, page_size=page_size)
        print("Запрос выполнен")

    except Exception as error:
        # Если произошла ошибка при выполнении запроса, выводим тип ошибки и ее описание,
        # а также запрос, который был выполнен до ошибки.
        print(f"{type(error).__name__}: {error}")
        print("Запрос:", cur.query)
        # Откатываем транзакцию и закрываем курсор.
        conn.rollback()
        cur.close()

    else:
        # Если запрос был успешно выполнен, фиксируем изменения в базе данных и закрываем курсор.
        conn.commit()

### Функция get_column_names принимает на вход следующие параметры:
### table - название таблицы в базе данных PostgreSQL
### cur - курсор для выполнения SQL-запросов

In [ ]:
# Функция get_column_names принимает на вход следующие параметры:
# table - название таблицы в базе данных PostgreSQL
# cur - курсор для выполнения SQL-запросов

def get_column_names(
    table: str,
    cur: psycopg2.extensions.cursor
) -> List[str]:
    # Выполняем SQL-запрос для получения списка названий столбцов таблицы table.
    # Используем INFORMATION_SCHEMA.COLUMNS, чтобы получить информацию о столбцах таблицы.
    # В запросе используем f-строку для подстановки названия таблицы.
    cursor.execute(
        f"SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{table}';")
    # Получаем результат выполнения запроса с помощью метода fetchall и сохраняем его в переменную col_names.
    # Используем list comprehension, чтобы получить список из первых элементов кортежей, содержащих названия столбцов.
    col_names = [result[0] for result in cursor.fetchall()]
    return col_names

### Функция принимает на вход следующие параметры:
### query - SQL-запрос для получения данных из базы данных PostgreSQL
### conn - соединение с базой данных
### cur - курсор для выполнения SQL-запросов
### df - фрейм данных pandas, в который будут добавлены извлеченные данные
### col_names - список названий столбцов таблицы, из которой извлекаются данные

In [ ]:
def get_data_from_db(
    query: str,
    conn: psycopg2.extensions.connection,
    cur: psycopg2.extensions.cursor,
    df: pd.DataFrame,
    col_names: List[str]
) -> pd.DataFrame:
    try:
        # В блоке try выполняется SQL-запрос с помощью метода execute курсора cur.
        cur.execute(query)
        # Затем в цикле while извлекаются данные из запроса по 100 строк за раз с помощью метода fetchmany.
        while True:
            # Разберем следующие 100 строк таблицы
            query_results = cur.fetchmany(100)
            # Если метод возвращает пустой список, значит, все данные были извлечены, и цикл прерывается.
            if query_results == list():
                break

            # Для каждой извлеченной строки создается словарь, в котором ключами являются названия столбцов таблицы,
            # а значениями - соответствующие значения из строки.
            # Созданный словарь добавляется в список results_mapped, который содержит все извлеченные строки.
            results_mapped = [
                {col_names[i]: row[i] for i in range(len(col_names))}
                for row in query_results
            ]

            # Список results_mapped добавляется в фрейм данных df с помощью метода append.
            # Параметр ignore_index=True указывает, что индексы строк должны быть пересчитаны.
            df = df.append(results_mapped, ignore_index=True)

        # Функция возвращает фрейм данных pandas с извлеченными данными.
        return df

    # В блоке except выводится сообщение об ошибке и откатывается транзакция.
    except Exception as error:
        print(f"{type(error).__name__}: {error}")
        print("Query:", cur.query)
        conn.rollback()

# Выполним подключение и создание базы данных

In [ ]:
# host, port, database, user, password
conn_info = load_connection_info("db.ini")

# Создаем желаемую базу данных
# Название базы данных записано в файле .ini

create_db(conn_info)  

In [ ]:
# Отобразим словарь из файла .ini
conn_info

# Выполним запрос на создание таблицы

In [ ]:
# Создаем подключение к созданной базе данных
connection = psycopg2.connect(**conn_info)
cursor = connection.cursor()

# Создание таблицы 
train_sql = """
    CREATE TABLE train_sound_DB (
        subject_id INTEGER,
        jitter_local NUMERIC,
        jitter_local_abs NUMERIC,
        jitter_rap NUMERIC,
        jitter_ppq5 NUMERIC,
        jitter_ddp NUMERIC,
        shimmer_local NUMERIC,
        shimmer_local_db NUMERIC,
        shimmer_apq3 NUMERIC,
        shimmer_apq5 NUMERIC,
        shimmer_apq11 NUMERIC,
        shimmer_dda NUMERIC,
        AC NUMERIC,
        NTH NUMERIC,
        HTN NUMERIC,
        median_pitch NUMERIC,
        mean_pitch NUMERIC,
        standard_deviation NUMERIC,
        minimum_pitch NUMERIC,
        maximum_pitch NUMERIC,
        number_of_pulses NUMERIC,
        number_of_periods NUMERIC,
        mean_period NUMERIC,
        standard_deviation_of_period NUMERIC,
        fraction_of_locally_unvoiced_frames NUMERIC,
        number_of_voice_breaks NUMERIC,
        degree_of_voice_breaks NUMERIC,
        UPDRS NUMERIC,
        class_information INTEGER
);

    """
create_table(train_sql, connection, cursor)

In [ ]:
# Закрываем соединение с базой данных
connection.close()
cursor.close()

In [ ]:
# Читаем файл с разделителем ","
df = pd.read_csv('train_data.txt', delimiter=',', header=None)

# Выводим первые 5 строк без столбца с номерами строк
print(df.head(5).to_string(index=False, header=None))

In [ ]:
# Создаем подключение к созданной базе данных
connection = psycopg2.connect(**conn_info)
cursor = connection.cursor()

# Выполняем запрос на добавление данных из ds
data_query = "INSERT INTO train_sound_DB VALUES %s"
insert_data(data_query, connection, cursor, df, 100)

# Закрываем соединение с базой данных
connection.close()
cursor.close()